In [1]:
import numpy as np
import pandas as pd

In [2]:
dir ='/root/JupyterNotebook/kaggle-Redefining-Cancer-Treatment'

In [3]:
train_cleantxt = pd.read_csv(dir+'/5 middle for stage2/train_cleantxt.txt')
test_cleantxt = pd.read_csv(dir+'/5 middle for stage2/test_cleantxt.txt')

In [4]:
train_len = len(train_cleantxt)
print(train_len)

3689


In [5]:
df_all = pd.concat((train_cleantxt, test_cleantxt), axis=0, ignore_index=True)

In [6]:
df_all.tail()

,ID,CleanText
4303,982,iκb kinase ikkβ involve tumor development prog...
4304,983,identification subtype-specific translocation ...
4305,984,cardio-facio-cutaneous cfc syndrome mim congen...
4306,985,screen tumor suppressor gene breast cancer rev...
4307,986,histiocytic neoplasm clonal hematopoietic diso...


In [7]:
variation_done_all = pd.read_csv(dir+'/5 middle for stage2/var_done.csv', )

In [8]:
variation_done_all['CleanText'] = df_all['CleanText']

In [9]:
df_pre_all = variation_done_all.drop(['Class'], axis=1)

In [10]:
df_pre_all['Gene'] = df_pre_all['Gene'].map(str.lower)
df_pre_all['Variation'] = df_pre_all['Variation'].map(str.lower)


In [11]:
df_pre_all.head()

,ID,Gene,Variation,first_letter,last_letter,location_number,operation,other_text,CleanText
0,0,fam58a,truncating mutations,none,none,0,truncating,none,cyclin-dependent kinase cdks regulate variety ...
1,1,cbl,w802*,w,none,802,substitution,none,abstract background non-small cell lung cancer...
2,2,cbl,q249e,q,e,249,substitution,none,abstract background non-small cell lung cancer...
3,3,cbl,n454d,n,d,454,substitution,none,recent evidence demonstrate acquire uniparenta...
4,4,cbl,l399v,l,v,399,substitution,none,oncogenic mutation monomeric casitas b-lineage...


In [12]:
df_pre_all.tail()

,ID,Gene,Variation,first_letter,last_letter,location_number,operation,other_text,CleanText
4303,982,tp63,s580p,s,p,580,substitution,none,iκb kinase ikkβ involve tumor development prog...
4304,983,scn4a,r672g,r,g,672,substitution,none,identification subtype-specific translocation ...
4305,984,braf,n581h,n,h,581,substitution,none,cardio-facio-cutaneous cfc syndrome mim congen...
4306,985,tshr,s281n,s,n,281,substitution,none,screen tumor suppressor gene breast cancer rev...
4307,986,ass1,s180n,s,n,180,substitution,none,histiocytic neoplasm clonal hematopoietic diso...


In [13]:
# label encoder
from sklearn import preprocessing
for c in df_pre_all.columns:
    if df_pre_all[c].dtype == 'object':
        if c == 'Gene':
            lbl = preprocessing.LabelEncoder()
            df_pre_all[c+'_lbl_enc'] = lbl.fit_transform(df_pre_all[c].values)  
            df_pre_all[c+'_len'] = df_pre_all[c].map(lambda x: len(str(x)))
        elif c == 'Variation':
            lbl = preprocessing.LabelEncoder()
            df_pre_all[c+'_lbl_enc'] = lbl.fit_transform(df_pre_all[c].values)  
            df_pre_all[c+'_len'] = df_pre_all[c].map(lambda x: len(str(x)))
            df_pre_all[c+'_words'] = df_pre_all[c].map(lambda x: len(str(x).split(' ')))
        elif c in ['CleanText','other_text']: 
            df_pre_all[c+'_len'] = df_pre_all[c].map(lambda x: len(str(x)))
            df_pre_all[c+'_words'] = df_pre_all[c].map(lambda x: len(str(x).split(' '))) 

In [18]:
def save(filename, contents): 
    fh = open(filename, 'w') 
    fh.write(contents) 
    fh.close() 

In [16]:
#one-hot for var-first and last letter,and operation
df_pre_all = pd.concat([df_pre_all, pd.get_dummies(df_pre_all['first_letter']).\
                        rename(columns=lambda x: 'firstletter_' + str(x))], axis=1)

In [17]:
df_pre_all = pd.concat([df_pre_all, pd.get_dummies(df_pre_all['last_letter']).\
                        rename(columns=lambda x: 'lastletter_' + str(x))], axis=1)

In [18]:
df_pre_all = pd.concat([df_pre_all, pd.get_dummies(df_pre_all['operation']).\
                        rename(columns=lambda x: 'operation_' + str(x))], axis=1)

In [20]:
df_pre_all['Gene_count'] = df_pre_all.apply(lambda x: str(x['CleanText']).count(str(x['Gene'])), axis=1)

In [21]:
df_pre_all['var_count'] = df_pre_all.apply(lambda x: str(x['CleanText']).count(str(x['first_letter'])+\
                                                            str(x['location_number'])+str(x['last_letter'])), axis=1)

In [22]:
df_pre_all['font_var_count'] = df_pre_all.apply(lambda x: str(x['CleanText']).count(str(x['first_letter'])+\
                                                            str(x['location_number'])), axis=1)

In [23]:
df_pre_all['last_var_count'] = df_pre_all.apply(lambda x: str(x['CleanText']).count(str(x['location_number'])+\
                                                            str(x['last_letter'])), axis=1)

In [24]:
# get operation count
truncating_list = ['trunc']
delins_list = ['insertion/deletion','deletion/insertion','delins']
fusion_list = ['fusion']
deletion_list = ['deletion']
ins_list = ['insertion']
amplification_list = ['amplifica']
overexpression_list = ['overexpress']
splice_list = ['splice']
duplication_list = ['duplicat']
substitution_list = ['substitut']

operation_dict = {'truncating':truncating_list, 'delins':delins_list,'fusion':fusion_list, \
                  'deletion':deletion_list, 'insertion':ins_list, 'amplification':amplification_list, \
'overexpression':overexpression_list,'splice':splice_list,'duplication':duplication_list,'substitution':substitution_list}

def get_oper_count(doc, operation):
    count = 0
    if operation != 'other':
        for x in operation_dict[operation]:
            count += doc.count(x)
    return count

In [25]:
df_pre_all['oper_count'] = df_pre_all.apply(lambda x: get_oper_count(str(x['CleanText']),str(x['operation'])), axis=1)

In [26]:
# get othertxt count

def get_othertxt_count(doc,othertxt):
    count = 0
    if othertxt != 'none':
        for x in othertxt.split(' '):
            count += doc.count(x)
    return count

In [27]:
df_pre_all['other_count'] = df_pre_all.apply(lambda x: get_othertxt_count(str(x['CleanText']),str(x['other_text'])), axis=1)

In [15]:
# combine var
def combinevar(first,middle,last):
    if middle == 0:
        return 'none'
    elif last == 'none':
        return str(first)+str(middle)
    elif first == 'none':
        return 'null'+str(middle)+str(last)
    else:
        return str(first)+str(middle)+str(last)
    

In [16]:
df_pre_all['temp_var'] = df_pre_all.apply(lambda x: combinevar(x['first_letter'],x['location_number'],x['last_letter']), axis=1)

In [17]:
#gen_var_lst  in train 
gen_var_lst = sorted(list(df_pre_all[:train_len].Gene.unique()) + list(df_pre_all[:train_len].temp_var.unique()))
print(len(gen_var_lst))

3137


In [18]:
gen_var_lst.remove('none')
print(len(gen_var_lst))

3136


In [61]:
gen_var_str_convert = ' '.join(gen_var_lst)
def save(filename, contents): 
    fh = open(filename, 'w') 
    fh.write(contents) 
    fh.close() 
save(dir+'/5 middle for stage2/gen_var_str.txt', gen_var_str_convert) 

In [19]:
import pickle
pickle.dump(gen_var_lst, open(dir+'/5 middle for stage2/'+'gen_var_lst', 'wb'))

In [63]:
df_done = df_pre_all.drop(['temp_var','ID','Gene','Variation','first_letter','last_letter','operation',\
                          'other_text'],axis=1)
df_done.tail()

,location_number,CleanText,Gene_lbl_enc,Gene_len,Variation_lbl_enc,Variation_len,Variation_words,other_text_len,other_text_words,CleanText_len,...,operation_overexpression,operation_splice,operation_substitution,operation_truncating,Gene_count,var_count,font_var_count,last_var_count,oper_count,other_count
4303,580,iκb kinase ikkβ involve tumor development prog...,381,4,3220,5,1,4,1,40417,...,0,0,1,0,0,63,63,63,2,0
4304,672,identification subtype-specific translocation ...,320,5,2919,5,1,4,1,40554,...,0,0,1,0,0,101,101,101,1,0
4305,581,cardio-facio-cutaneous cfc syndrome mim congen...,50,4,2162,5,1,4,1,46680,...,0,0,1,0,144,0,6,0,17,0
4306,281,screen tumor suppressor gene breast cancer rev...,386,4,3141,5,1,4,1,17892,...,0,0,1,0,0,28,28,28,0,0
4307,180,histiocytic neoplasm clonal hematopoietic diso...,24,4,3071,5,1,4,1,40515,...,0,0,1,0,0,57,57,57,1,0


In [64]:
import pickle
pickle.dump(df_done, open(dir+'/5 middle for stage2/'+'onehotvar_cleantxt_data', 'wb'))